In [4]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt

In [5]:
dg_df = pd.read_csv('../big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/Demographics.csv')
dg_df.head(16)

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7
5,2,MALE,5.6
6,6,FEMALE,5.8
7,7,FEMALE,5.3
8,8,FEMALE,5.6
9,10,FEMALE,6.0


In [6]:
# 연속적인 결측치 구간 식별
missing_ranges = []

for id in range(1, len(dg_df)+1):
    if id == 15:
        continue
    df = pd.read_csv(f'../big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2/{id:03}/HR_{id:03}.csv')

    print(f'[ID: {id}] Now processing ...')

    # 데이터 처리
    df['datetime'] = pd.to_datetime(df['datetime'])
    # 전체 시간 범위를 250ms 간격으로 생성
    full_time_range = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq='15.625 ms')

    # 누락된 시간대 찾기
    missing_times = full_time_range.difference(df['datetime'])

    if not missing_times.empty:
        start_time = missing_times[0]
        for i in range(1, len(missing_times)):
            if missing_times[i] != missing_times[i-1] + pd.Timedelta('15.625 ms'):
                end_time = missing_times[i-1]
                #missing_ranges.append((start_time, end_time, (end_time - start_time) / pd.Timedelta('15.625 ms') + 1))
                missing_ranges.append({
                    'id': id,
                    'start': start_time,
                    'end': end_time,
                    'count': int((end_time - start_time) / pd.Timedelta('15.625 ms') + 1)
                })
                start_time = missing_times[i]
        # 마지막 구간 추가
        end_time = missing_times[-1]
        #missing_ranges.append((start_time, end_time, (end_time - start_time) / pd.Timedelta('15.625 ms') + 1))
        missing_ranges.append({
            'id': id,
            'start': start_time,
            'end': end_time,
            'count': int((end_time - start_time) / pd.Timedelta('15.625 ms') + 1)
        })

# 결과를 데이터프레임으로 변환
missing_ranges_df = pd.DataFrame(missing_ranges)

# CSV 파일로 저장
missing_ranges_df.to_csv('hr_missing_ranges.csv', index=False)

[ID: 1] Now processing ...


/tmp/ipykernel_267694/3507519049.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'])


KeyboardInterrupt: 